## Import the Library Components

In [22]:
!pip install highcharts-core -q

In [23]:
from highcharts_core.chart import Chart
from highcharts_core.options import HighchartsOptions
from highcharts_core.options.plot_options import PlotOptions
from highcharts_core.options.axes.x_axis import XAxis
from highcharts_core.options.axes.y_axis import YAxis
from highcharts_core.options.axes.accessibility import AxisAccessibility
from highcharts_core.options.axes.title import AxisTitle
from highcharts_core.options.title import Title
from highcharts_core.options.subtitle import Subtitle
from highcharts_core.options.legend import Legend
from highcharts_core.options.plot_options.series import SeriesOptions
from highcharts_core.options.series.area import LineSeries
from highcharts_core.options.series.labels import SeriesLabel
from highcharts_core.options.responsive import Responsive, ResponsiveRules, Condition
from highcharts_core.constants import EnforcedNull

## Instantiate the Preliminary Chart Options

In [24]:
chart_options = HighchartsOptions(
    title = Title(text = 'U.S Solar Employment Growth by Job Category, 2010-2020',
                  align = 'left'),
    subtitle = Subtitle(text = 'Source: <a href="https://irecusa.org/programs/solar-jobs-census/" target="_blank">IREC</a>',
                        align = 'left'),
    y_axis = YAxis(title = AxisTitle(text = 'Number of Employees')),
    x_axis = XAxis(
        accessibility = AxisAccessibility(range_description = 'Range: 2010 to 2020')
    ),
    legend = Legend(layout = 'vertical',
                    align = 'right',
                    vertical_align = 'middle'),
    plot_options = PlotOptions(series = SeriesOptions(point_start = 2010,
                                                      label = SeriesLabel(connector_allowed = False)))
)

## Instantiate the Responsive Configuration

In [25]:
override_options = HighchartsOptions(legend = Legend(layout = 'horizontal',
                                                     align = 'center',
                                                     vertical_align = 'bottom'))
responsive_config = Responsive(
    rules = [
        ResponsiveRules(chart_options = override_options,
                        condition = Condition(max_width = 500))
    ]
)
chart_options.responsive = responsive_config

## Instantiate the Charts' Series (plural)

In [26]:
series1 = LineSeries(name = 'Installation & Developers',
                     data = [43934, 48656, 65165, 81827, 112143, 142383, 171533, 165174, 155157, 161454, 154610])
series2 = LineSeries(name = 'Manufacturing',
                     data = [24916, 37941, 29742, 29851, 32490, 30282, 38121, 36885, 33726, 34243, 31050])
series3 = LineSeries(name = 'Sales & Distribution',
                     data = [11744, 30000, 16005, 19771, 20185, 24377, 32147, 30912, 29243, 29213, 25663])
series4 = LineSeries(name = 'Operations & Maintenance',
                     data = [EnforcedNull, EnforcedNull, EnforcedNull, EnforcedNull, EnforcedNull, EnforcedNull, EnforcedNull, EnforcedNull, 11164, 11218, 10077])
series5 = LineSeries(name = 'Other',
                     data = [21908, 5548, 8105, 11248, 8989, 11816, 18274, 17300, 13053, 11906, 10073])

chart_options.add_series(series1, series2, series3, series4, series5)

## Instantiate and Display the Chart

In [27]:
chart = Chart.from_options(chart_options)
chart.display()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [44]:
as_dict = {
    'chart': {
        'type': 'spline',
        'inverted': True
    },
    'title': {
        'text': 'Atmosphere Temperature by Altitude',
        'align': 'left'
    },
    'subtitle': {
        'text': 'According to the Standard Atmosphere Model',
        'align': 'left'
    },
    'xAxis': {
        'reversed': False,
        'title': {
            'enabled': True,
            'text': 'Altitude'
        },
        'labels': {
            'format': '{value} km'
        },
        'accessibility': {
            'rangeDescription': 'Range: 0 to 80 km.'
        },
        'maxPadding': 0.05,
        'showLastLabel': True
    },
    'yAxis': {
        'title': {
            'text': 'Temperature'
        },
        'labels': {
            'format': '{value}°'
        },
        'accessibility': {
            'rangeDescription': 'Range: -90°C to 20°C.'
        },
        'lineWidth': 2
    },
    'legend': {
        'enabled': False
    },
    'tooltip': {
        'headerFormat': '<b>{series.name}</b><br/>',
        'pointFormat': '{point.x} km: {point.y}°C'
    },
    'plotOptions': {
        'spline': {
            'marker': {
                'enable': False
            }
        }
    },
    'series': [{
        'name': 'Temperature',
        'data': [[0, 15], [10, -50], [20, -56.5], [30, -46.5], [40, -22.1],
            [50, -2.5], [60, -27.7], [70, -55.7], [80, -76.5]]
    }]
}

In [45]:
chart = Chart(options = as_dict)
chart.display()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [68]:
def extract_chart_data(chart):
    opts = chart.options.to_dict()
    return {
        "series": opts.get("series", []),
        "categories": opts.get("xAxis", {}).get("categories", []),
        "title": opts.get("title", {}).get("text", ""),
        "subtitle": opts.get("subtitle", {}).get("text", ""),
        "type": opts.get("chart", {}).get("type", ""),  # <-- FIXED
    }


In [69]:
def build_summarization_prompt(chart_json, user_question=None):
    system_prompt = (
        "You are a senior analytics assistant that interprets and summarizes chart data with precision. "
        "Your job is to identify key trends, insights, growth rates (e.g., YoY, MoM), anomalies, comparisons, and business implications. "
        "Use clear, structured language and support all conclusions with data points from the chart."
    )

    base = (
        f"Title: {chart_json.get('title', '')}\n"
        f"Subtitle: {chart_json.get('subtitle', '')}\n"
        f"Chart Type: {chart_json.get('type', '')}\n"
        f"Categories (typically X-axis): {chart_json.get('categories', [])}\n\n"
        f"Series Data:\n"
    )

    for series in chart_json['series']:
        base += f"- Series Name: {series.get('name', 'Unnamed')}, Values: {series.get('data', [])}\n"

    default_structure = (
        "Analyze the chart data and provide insights using the following structure:\n"
        "1. Key Trend Summary (e.g., increasing, seasonal, declining)\n"
        "2. Series Comparison (e.g., which series outperformed others?)\n"
        "3. Growth Metrics (e.g., YoY, MoM % growth)\n"
        "4. Anomalies/Spikes (e.g., unexpected dips or peaks)\n"
        "5. Recommendation or business implication (if any)\n\n"
        "Respond using bullet points. Support each insight with relevant data values from the chart."
    )

    if user_question:
        user_question = f"{user_question}\n\nFollow this structure if applicable:\n{default_structure}"
    else:
        user_question = default_structure

    full_prompt = f"{base}\n\nTask: {user_question}\nAnswer:"
    return system_prompt, full_prompt


In [70]:
import requests

def query_grok(system_prompt, user_prompt, api_key):
    url = "https://api.groq.com/openai/v1/chat/completions"  
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "llama3-70b-8192",  
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        "temperature": 0.7,
        "max_tokens": 800
    }

    response = requests.post(url, headers=headers, json=payload)
    return response.json()["choices"][0]["message"]["content"]


In [71]:
from highcharts_core.chart import Chart
import json

chart_json = extract_chart_data(chart)
api_key = "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"

In [72]:
custom_question = "Summarize the graph and give me top trends"
sys_prompt, user_prompt = build_summarization_prompt(chart_json, custom_question)
summary = query_grok(sys_prompt, user_prompt, api_key)

print("=== SUMMARY ===")
print(summary)

=== SUMMARY ===
Here is the summary of the chart data:

• **Key Trend Summary**: The atmosphere temperature decreases as altitude increases, with some fluctuations in between.

• **Series Comparison**: Since there is only one series in the chart, no comparison can be made.

• **Growth Metrics**: Not applicable, as this chart does not depict growth metrics.

• **Anomalies/Spikes**: There are two anomalies/spikes in the chart:
  - A sudden drop in temperature from 15°C to -50°C between 0 and 10 km altitude.
  - A local minimum at 20 km altitude with a temperature of -56.5°C, which is lower than the surrounding points.

• **Recommendation or business implication**: This chart suggests that the temperature in the atmosphere decreases as you move higher in altitude, which is a well-known scientific phenomenon. This information can be useful in various fields such as aviation, aerospace engineering, and climate modeling.
